In [4]:
import tweepy as TW
import pandas as pd
import nltk
import re

# connect to twitter API

CONSUMER_KEY = 'XkmU3czKdJ1jfzUAIWELFC9y8'
CONSUMER_SECRET = 'HlxpwfuShIh8OXRGjWgT2h3fD9JMC6HNtQ11N28WFFDzpCUHO0'
ACCESS_TOKEN = 	'4826647649-GRcsvzCoPjdTdosCCgYY9zalhBytNbSuVaemIvH'
ACCESS_TOKEN_SECRET = '93l4Mh1KLfexPNxwqZ4mtjUhseGWe5xQecS0Sd7YfdCbm'

auth =TW.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)  
api = TW.API(auth)

# Load data in Dataframe with Twitter API
data = {'date': [], 'text': []}

for tweet in TW.Cursor(api.search, 
                    q="blockchain",                       
                    since="2017-06-10", 
                    until="2017-08-10", 
                    lang="en").items(100):   
    
        #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
        data['text'].append( tweet.text)
        data['date'].append(tweet.created_at)
#Оберточка дикт в пандас
data = pd.DataFrame(data)

In [5]:
data.text = data.text.drop_duplicates()
data = data.dropna()
print()

## Filter dataframe

In [7]:
stop_list = nltk.corpus.stopwords.words('english') + ["rt"] + ['[slides]']+ ["re"]

lemmatizer = nltk.stem.WordNetLemmatizer()
reg = '(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)'

emoji_pattern = re.compile(u'('
    u'\ud83c[\udf00-\udfff]|'
    u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
    u'[\u2600-\u26FF\u2700-\u27BF])+', 
    re.UNICODE)

classifier =[]
def preprocess(tweet):   

    tweet = tweet.lower()
    #tweet = " ".join(tweet.split('#'))
    tweet = re.sub('((www\.[^\s]+)|(https://[^\s]+))','',tweet)
    tweet = re.sub('((pic\.[^\s]+)|(https://[^\s]+))','',tweet)
    tweet = re.sub("(http\S+)|(https\S+)", '', tweet)
    tweet = re.sub(reg,'',tweet)
    #tweet = re.sub(u'[a-zA-Z0-9./]+\.[a-zA-Z0-9./ ]+.*$','',tweet)
    
    tweet = re.sub(u'[\(\)]',' ',tweet)
    tweet = re.sub(u'\n',' ',tweet)
    tweet = re.sub(u'@[^\s]+','',tweet)  
    tweet = re.sub(r'#[^\s]+', '', tweet)
    tweet = tweet.strip('\'"')
    twitter = emoji_pattern.sub('', tweet)
    tweet = re.sub('[\W]', ' ', tweet)
    
    # Removing stop words 
    tweet  = " ".join([word for word in tweet.split(" ") if word not in stop_list])
    tweet = " ".join([lemmatizer.lemmatize(word) for word in tweet.split(" ")])


    return tweet


In [12]:
data['processed_text'] = data.text.apply(preprocess)

# Semantic analise

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
s_analisys_dataframe = pd.DataFrame()

for i in data.index:
    ss = sid.polarity_scores(data.processed_text[i])
    ss['date'] = data.date[i]
    returnDF = pd.DataFrame(pd.Series(ss,index=ss.keys())).T
    s_analisys_dataframe = s_analisys_dataframe.append(returnDF, ignore_index=True)

## Load data